In [81]:
from bs4 import BeautifulSoup
import requests
import time
import pandas as pd

In [7]:
headers = {
    'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10.12; rv:55.0) Gecko/20100101 Firefox/55.0',
}

In [ ]:
list_resultado = []
for i in range(1, 13965):
    try:
        if i % 50 == 0:
            print(i)
            time.sleep(5)
        URL = f'https://mf-rg.intusvidere.com.br/credor1_evolqgc.php?id={i}'
        response = requests.get(URL, headers=headers).content
        soup     = BeautifulSoup(response, 'html.parser')

        containers = soup.find_all('div', class_='row')[0].find_all('div')
        cpf        = containers[0].text.replace('CPF-CNPJ:', '').replace('.','').replace('-','').strip()
        nome       = containers[1].text.replace('Nome:', '').strip()
        empresa    = containers[2].text.replace('Empresa:', '').strip()

        tabelas                    = soup.find_all('table')
        valores                    = tabelas[1].find_all('tr')[2].find_all('th')
        empresa_2010               = valores[0].text
        credito_classe1_2010_valor = valores[1].text
        credito_classe3_2010_valor = valores[2].text
        credito_2010_total         = valores[3].text
        credito_classe1_2010_faixa = 'Até 150SM'
        credito_classe3_2010_faixa = '> 150SM'
        reserva_classe1_2010_faixa = 'Até 150SM'
        reserva_classe3_2010_faixa = '> 150SM'
        reserva_classe1_2010_valor = valores[4].text
        reserva_classe3_2010_valor = valores[5].text
        reserva_2010_total         = valores[6].text
        valores                    = tabelas[3].find_all('tr')[2].find_all('th')
        empresa_2024               = valores[0].text
        credito_classe1_2024_valor = valores[1].text
        credito_classe3_2024_valor = valores[2].text
        credito_2024_total         = valores[3].text
        credito_classe1_2024_faixa = 'Até 150SM'
        credito_classe3_2024_faixa = '> 150SM'
        reserva_classe1_2024_faixa = 'Até 150SM'
        reserva_classe3_2024_faixa = '> 150SM'
        reserva_classe1_2024_valor = valores[4].text
        reserva_classe3_2024_valor = valores[5].text
        reserva_2024_total         = valores[6].text
        observacao                 = soup.find_all('div', class_='border-danger')[0].text

        dict_divida = {
            'cpf'                       : cpf,
            'nome'                      : nome,
            'empresa'                   : empresa,
            'empresa_2010'              : empresa_2010,
            'credito_classe1_2010_valor': credito_classe1_2010_valor,
            'credito_classe3_2010_valor': credito_classe3_2010_valor,
            'credito_2010_total'        : credito_2010_total,
            'credito_classe1_2010_faixa': 'Até 150SM',
            'credito_classe3_2010_faixa': '> 150SM',
            'reserva_classe1_2010_faixa': 'Até 150SM',
            'reserva_classe3_2010_faixa': '> 150SM',
            'reserva_classe1_2010_valor': reserva_classe1_2010_valor,
            'reserva_classe3_2010_valor': reserva_classe3_2010_valor,
            'reserva_2010_total'        : reserva_2010_total,
            'empresa_2024'              : empresa_2024,
            'credito_classe1_2024_valor': credito_classe1_2024_valor,
            'credito_classe3_2024_valor': credito_classe3_2024_valor,
            'credito_2024_total'        : credito_2024_total,
            'credito_classe1_2024_faixa': 'Até 150SM',
            'credito_classe3_2024_faixa': '> 150SM',
            'reserva_classe1_2024_faixa': 'Até 150SM',
            'reserva_classe3_2024_faixa': '> 150SM',
            'reserva_classe1_2024_valor': reserva_classe1_2024_valor,
            'reserva_classe3_2024_valor': reserva_classe3_2024_valor,
            'reserva_2024_total'        : reserva_2024_total,
            'observacao'                : soup.find_all('div', class_='border-danger')[0].text.replace('\n', ' ')
        }
        list_resultado.append(dict_divida)
    except Exception as e:
        print(f'erro: {e}', i)

df_result = pd.DataFrame(list_resultado)
df_result.to_csv('scraping_result.csv')

In [88]:
import sqlalchemy

In [86]:
class AcruxDBDataLake:
    
    def __init__(self):
        self.host     = '35.237.180.90'
        self.database = 'prd-parceiros'
        self.user     = 'data_user'
        self.password = "]_;>K%*E}Mc/'3:u"
        
    
    def conecta_insert(self):
        conn = sqlalchemy.create_engine(f"mysql+mysqlconnector://{self.user}:{self.password}@{self.host}:3306/{self.database}")
        return conn

In [91]:
df_result['observacoes'] = df_result['observacao']

In [95]:
df_result = df_result.drop('observacao', axis=1)

In [87]:
db = AcruxDBDataLake()

In [96]:
dtype = {
        "cpf"    : sqlalchemy.VARCHAR(20),
        'nome'   : sqlalchemy.VARCHAR(250),
        'empresa': sqlalchemy.VARCHAR(100),
        'credito_classe1_2010_faixa': sqlalchemy.VARCHAR(100),
        'credito_classe1_2010_valor': sqlalchemy.VARCHAR(100),
        'credito_classe3_2010_faixa': sqlalchemy.VARCHAR(100),
        'credito_classe3_2010_valor': sqlalchemy.VARCHAR(100),
        'credito_2010_total'        : sqlalchemy.VARCHAR(100),
        'empresa_2010'              : sqlalchemy.VARCHAR(100),
        'reserva_classe1_2010_faixa': sqlalchemy.VARCHAR(100),
        'reserva_classe1_2010_valor': sqlalchemy.VARCHAR(100),
        'reserva_classe3_2010_faixa': sqlalchemy.VARCHAR(100),
        'reserva_classe3_2010_valor': sqlalchemy.VARCHAR(100),
        'reserva_2010_total'        : sqlalchemy.VARCHAR(100),
        'credito_classe1_2024_faixa': sqlalchemy.VARCHAR(100),
        'credito_classe1_2024_valor': sqlalchemy.VARCHAR(100),
        'credito_classe3_2024_faixa': sqlalchemy.VARCHAR(100),
        'credito_classe3_2024_valor': sqlalchemy.VARCHAR(100),
        'credito_2024_total'        : sqlalchemy.VARCHAR(100),
        'empresa_2024'              : sqlalchemy.VARCHAR(100),
        'reserva_classe1_2024_faixa': sqlalchemy.VARCHAR(100),
        'reserva_classe1_2024_valor': sqlalchemy.VARCHAR(100),
        'reserva_classe3_2024_faixa': sqlalchemy.VARCHAR(100),
        'reserva_classe3_2024_valor': sqlalchemy.VARCHAR(100),
        'reserva_2024_total'        : sqlalchemy.VARCHAR(100),
        'observacoes'               : sqlalchemy.VARCHAR(250)
    }


conn = db.conecta_insert()

df_result.to_sql(
                    'whitelabel_massasfalidas',
                    conn,
                    if_exists='append',
                    index=False,
                    chunksize=500,
                    dtype=dtype
                    )

13964